In [1]:
from prep_data import PrepData
from anchor_boxes import *
from yolo_model import *
from predicting import *

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import model_from_json

In [5]:
input_shape = (676, 380, 3)
grid_dim = 13
anchor_sizes = [(0.2, 0.1), (0.3, 0.2), (0.4, 0.3), (0.5, 0.4)]
num_anchors = len(anchor_sizes)

In [6]:
p = PrepData()
train, test = p.load_images()
labels = p.connect_labels(train, grid_dim, anchor_sizes, iou_vectorized)
train, test = p.preprocess_images(train), p.preprocess_images(test)

# swap axes of x and y of the image
train = np.swapaxes(train, 1, 2)
test = np.swapaxes(test, 1, 2)
train.shape

(1001, 676, 380, 3)

In [2]:
def custom_iou_loss(y_true, y_pred):
    # Compute IOUs
    ious = iou_vectorized(y_true[..., :4], y_pred[..., :4])
    iou_loss = tf.abs(ious - y_true[..., -1]) # IOU losses - the confidence scores
    
    # Mean squared error over IOU losses
    return tf.reduce_mean(tf.square(iou_loss))

def custom_iou_metric(y_true, y_pred):
    ious = iou_vectorized(y_true[..., :4], y_pred[..., :4])
    return tf.reduce_mean(ious)

In [26]:
model = yolo_model(input_shape, grid_dim, num_anchors)
model.compile(optimizer='adam', loss=custom_iou_loss, metrics=['accuracy', custom_iou_metric])
model.fit(train, labels, epochs=1, batch_size=16, verbose=1)

/Users/jadenvanrijswijk/Code/Car Pose Estimation/.venv/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


63/63 ━━━━━━━━━━━━━━━━━━━━ 181s 3s/step - accuracy: 0.0453 - custom_iou_metric: 1.2937e-05 - loss: 8.4267e-07


In [28]:
# Save the model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.weights.h5")

Model saved to disk


In [3]:
# Load the model
json_file = open('./model/model.json', 'r') 
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json, custom_objects={'custom_iou_loss': custom_iou_loss, 'custom_iou_metric': custom_iou_metric})
model.load_weights("./model/model.weights.h5") # set weights

In [7]:
image = test[0].tolist()
predicted_image = predict_image(image, model, grid_dim, iou_vectorized)

# show image until user presses a key
cv2.imshow('image', predicted_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

I0000 00:00:1712596803.575366 40835845 service.cc:145] XLA service 0x2d42a1870 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1712596803.576093 40835845 service.cc:153]   StreamExecutor device (0): Host, Default Version
2024-04-08 19:20:03.617114: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1712596804.004993 40835845 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
